## Steps to address Aaron’s surprise that SV-predictor is so much worse than SNV-predictor at predicting coding constraint


In [ ]:
# 1. let genes (critical and olfactory) be the "large" windows (they are now of varying sizes)
# 2. get their exon coordinates (together with parent genes)
# 3. assign z-scores to exons, group by parent genes, aggregate across exons, and get z-scores for genes 
# 4. remove large deletions, intersect deletions with exons (not introns), and possibly make -f parameter more stringent, and include -r bedtools parameter, group by parent genes, aggregate across exons, to get deletion count for each gene
# 5. normalize deletion count by total exon length, so that we can compare across genes
# 6. re-run PR curves 

# 7. Visually inspect overlap of deletions with positive examples
